In [2]:
import pandas as pd
import os.path as osp
from pathlib import Path
import numpy as np
import os
from scipy.io import loadmat
from collections import defaultdict
from sklearn.model_selection import StratifiedGroupKFold
import shutil
from sklearn.model_selection import train_test_split

In [92]:
import os

In [100]:
subj_ids =[]
for subj in os.listdir("../datasets/alexeev/fmri/raw/aal"):
    if not ("embed" in subj  or ".ipynb_checkpoints" in subj):
        subj_ids.append(Path(subj).stem)

In [101]:
data_morph =pd.read_csv('../datasets/alexeev/dataset.csv')

In [104]:
data_morph.loc[data_morph['ID'].isin(subj_ids)].to_csv('../datasets/alexeev/dataset.csv', index=False)

In [ ]:
subj_ids

In [91]:
data_morph['ID']

0      sub-0085
1      sub-0091
2      sub-0046
3      sub-0052
4      sub-0114
         ...   
112    sub-0125
113    sub-0119
114    sub-0077
115    sub-0063
116    sub-0088
Name: ID, Length: 117, dtype: object

In [13]:
def return_correct_subject(index):
    index = str(index)
    if len(index) == 1:
        return "sub-000"+index
    elif len(index) == 2:
        return "sub-00"+index
    elif len(index) == 3:
        return "sub-0"+index
    else:
        return None

In [15]:
data_morph['ID'] =data_morph['ID'].apply(return_correct_subject)

In [26]:
data_morph.to_csv('../datasets/alexeev/dataset.csv')

In [77]:
data_morph

,Left-Lateral-Ventricle_NVoxels,Left-Inf-Lat-Vent_NVoxels,Left-Cerebellum-White-Matter_NVoxels,Left-Cerebellum-Cortex_NVoxels,Left-Thalamus-Proper_NVoxels,Left-Caudate_NVoxels,Left-Putamen_NVoxels,Left-Pallidum_NVoxels,3rd-Ventricle_NVoxels,4th-Ventricle_NVoxels,...,superiorparietal_CurvInd_lh,superiortemporal_CurvInd_lh,supramarginal_CurvInd_lh,frontalpole_CurvInd_lh,temporalpole_CurvInd_lh,transversetemporal_CurvInd_lh,insula_CurvInd_lh,file,ID,target
0,4641.0,127.0,12705.0,62431.0,7976.0,3750.0,4983.0,2055.0,860.0,1837.0,...,5.6,4.5,6.2,0.4,0.6,0.7,3.4,morphometry85,sub-0085,н
1,8319.0,197.0,22172.0,67420.0,7895.0,3535.0,5100.0,1887.0,1299.0,1760.0,...,10.6,6.8,10.9,1.1,1.4,0.5,5.5,morphometry91,sub-0091,н
2,5737.0,266.0,14301.0,55665.0,7745.0,3821.0,4533.0,2073.0,911.0,1492.0,...,7.4,4.0,6.0,0.7,1.0,0.5,4.2,morphometry46,sub-0046,ш
3,6896.0,152.0,14951.0,58844.0,8366.0,3313.0,4415.0,2447.0,877.0,1764.0,...,8.7,5.7,6.7,1.1,1.5,0.7,4.9,morphometry52,sub-0052,ш
4,4766.0,142.0,12010.0,53155.0,7435.0,3107.0,4999.0,2009.0,1007.0,1573.0,...,7.7,7.4,7.7,0.8,1.0,1.1,3.8,morphometry114,sub-0114,н
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,4648.0,271.0,14372.0,63467.0,7735.0,3138.0,4974.0,2019.0,726.0,1430.0,...,7.9,6.4,8.5,0.8,0.8,0.8,4.2,morphometry125,sub-0125,н
113,12214.0,423.0,14957.0,70766.0,8507.0,4167.0,5341.0,2448.0,1277.0,1919.0,...,9.6,6.9,6.1,0.7,3.2,0.6,6.0,morphometry119,sub-0119,н
114,4035.0,117.0,12931.0,58002.0,7399.0,3582.0,4883.0,2153.0,889.0,1763.0,...,7.8,3.5,5.5,0.6,1.2,0.5,4.3,morphometry77,sub-0077,н
115,7281.0,299.0,12633.0,55697.0,7429.0,3216.0,4542.0,1893.0,890.0,1782.0,...,5.9,3.1,4.5,0.5,0.7,0.4,3.1,morphometry63,sub-0063,шк


In [55]:
subj_id_col ='ID'
# fmri_id_col ='FILE_ID'
target_col ='target'
target_file = "dataset.csv"
morph_dataset_file ="dataset.csv"

In [61]:
def load_morphometry(  
        path):
        """ Load morphometry features vector 
            Args:
                path (str | Path): path to dit w/ connectivity matrices
                    and time series for each subject id. It's assumed that
                    each file corresponds to one sibject and one time of data.
            Return:
                Dict mapping subject_id to morphometris vector
        """
        morph_dataset = pd.read_csv(osp.join(path,morph_dataset_file))
        morp_map ={}
        for row in morph_dataset.iterrows():
            morp_map[row[1]['ID']] =row[1][1:-3].values
        
        return morp_map 

In [62]:
load_morphometry('../datasets/alexeev/')

{'sub-0085': array([4641.0, 127.0, 12705.0, 62431.0, 7976.0, 3750.0, 4983.0, 2055.0,
        860.0, 1837.0, 23437.0, 4272.0, 1637.0, 864.0, 333.0, 4257.0, 7.0,
        228.0, 4708.0, 129.0, 14013.0, 65419.0, 7796.0, 3776.0, 5097.0,
        1884.0, 4271.0, 1765.0, 504.0, 4261.0, 0.0, 309.0, 0.0, 804.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 109.0, 939.0, 619.0, 673.0, 692.0, 831.0,
        5111.4, 165.8, 13452.8, 62539.0, 7721.0, 3733.6, 5056.0, 2038.4,
        885.6, 1946.2, 23106.9, 4137.5, 1575.1, 875.7, 354.3, 3916.3, 15.1,
        216.0, 5067.0, 160.9, 14569.0, 65740.3, 7537.3, 3737.5, 5151.4,
        1824.1, 4166.1, 1736.1, 494.6, 3919.9, 0.0, 289.9, 0.0, 794.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 103.0, 848.1, 538.4, 577.5, 581.5, 771.9,
        44.278, 51.7638, 89.2952, 67.9476, 90.8553, 83.3699, 89.0076,
        100.3221, 46.8942, 38.0838, 77.2399, 74.8968, 73.2615, 60.6238,
        79.9039, 85.1663, 61.1429, 58.5088, 44.5308, 43.9845, 84.5234,
        66.458, 88.3966, 82.6692, 85.2825,

In [105]:
def load_targets(
        dataset_dir: str,
    ):
        """Load and process *cobre* targets"""

        target = pd.read_csv(osp.join(dataset_dir, target_file))
        target = target[[subj_id_col, target_col]]

        # check that there are no different labels assigned to the same ID
        max_labels_per_id = (
            target.groupby(subj_id_col)[target_col].nunique().max()
        )
        assert max_labels_per_id == 1, "Diffrent targets assigned to the same id!"

        # remove duplicates by subj_id
        target.drop_duplicates(subset=[subj_id_col], inplace=True)
        # set subj_id as index
        target.set_index(subj_id_col, inplace=True)

        # leave only Schizo and Control
        target = target[
            target[target_col].isin(("н", "ш"))
        ].copy()

        # label encoding
        label2idx: dict[str, int] = {
            x: i for i, x in enumerate(target[target_col].unique())
        }
        idx2label: dict[int, str] = {i: x for x, i in label2idx.items()}
        target[target_col] = target[target_col].map(label2idx)

        return target, label2idx, idx2label

In [106]:
target, label2idx, idx2label =load_targets('../datasets/alexeev/')

In [107]:
def load_cms(
        path) :

        """Load connectivity matrices, fMRI time series
        and mapping node idx -> ROI name.

        Maps sibj_id to CM and ts
        """

        path = Path(path)

        data = {}
        time_series = {}
        # ROI names, extacted from CMs
        roi_map: dict[int, str] = {}
        
        for p in path.glob("*.csv"):
            name = p.stem
            x = pd.read_csv(p)
            print(x.iloc[:,1:].values)

            values = x.iloc[:,1:].values.astype(np.float32)
            if p.stem.endswith("_embed"):
                time_series[name] = values
            else:
                data[name] = values
                if not roi_map:
                    roi_map = dict(enumerate(x.columns))

        return data, time_series, roi_map

In [108]:
data, ts, roi_map =load_cms("../datasets/alexeev/fmri/raw")

In [109]:
len(data)

0

In [110]:
def generate_splits(file_ids, subj_ids, y, seed: int = 1380):
    # split into train/test
    file_ids = np.array(file_ids)
    idx = np.arange(len(file_ids)) 
    
    train_idx, test_idx = train_test_split(
        idx, test_size=0.2, stratify=y, shuffle=True, random_state=seed,
    )

    train, y_train = subj_ids[train_idx], y[train_idx]
    test, _ = subj_ids[test_idx], y[test_idx]

    # split train into cv folds
    cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=seed)
    folds: dict[str, list] = {}
    for i, (train_fold, valid_fold) in enumerate(cv.split(train, y_train, subj_ids[train_idx])):
        folds[i]={
            'train': [str(k) for k in list(train.iloc[train_fold])],
            'valid': [str(k) for k in list(train.iloc[valid_fold])],
        }
    folds['test'] = [str(k) for k in list(test)]

    return folds

In [111]:
target

,target
ID,
sub-0085,0
sub-0091,0
sub-0046,1
sub-0052,1
sub-0114,0
...,...
sub-0124,0
sub-0001,1
sub-0125,0


In [75]:
list(target['target'].values)

[0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0]

In [74]:
list(target['target'].index)

['sub-0085',
 'sub-0091',
 'sub-0046',
 'sub-0052',
 'sub-0114',
 'sub-0100',
 'sub-0128',
 'sub-0101',
 'sub-0115',
 'sub-0047',
 'sub-0090',
 'sub-0084',
 'sub-0092',
 'sub-0086',
 'sub-0079',
 'sub-0051',
 'sub-0045',
 'sub-0117',
 'sub-0116',
 'sub-0102',
 'sub-0044',
 'sub-0050',
 'sub-0078',
 'sub-0087',
 'sub-0093',
 'sub-0097',
 'sub-0083',
 'sub-0054',
 'sub-0040',
 'sub-0106',
 'sub-0112',
 'sub-0113',
 'sub-0107',
 'sub-0041',
 'sub-0055',
 'sub-0082',
 'sub-0096',
 'sub-0080',
 'sub-0094',
 'sub-0043',
 'sub-0111',
 'sub-0105',
 'sub-0009',
 'sub-0104',
 'sub-0110',
 'sub-0042',
 'sub-0095',
 'sub-0019',
 'sub-0025',
 'sub-0031',
 'sub-0030',
 'sub-0024',
 'sub-0018',
 'sub-0032',
 'sub-0026',
 'sub-0027',
 'sub-0033',
 'sub-0037',
 'sub-0023',
 'sub-0036',
 'sub-0020',
 'sub-0034',
 'sub-0035',
 'sub-0021',
 'sub-0038',
 'sub-0010',
 'sub-0011',
 'sub-0039',
 'sub-0013',
 'sub-0012',
 'sub-0016',
 'sub-0017',
 'sub-0015',
 'sub-0029',
 'sub-0028',
 'sub-0014',
 'sub-0098',

In [112]:
target['ID'] =target.index.to_list()

In [113]:
new_folds =generate_splits(target['ID'], target['ID'], target['target'])

In [114]:
new_folds

{0: {'train': ['sub-0041',
   'sub-0032',
   'sub-0042',
   'sub-0055',
   'sub-0040',
   'sub-0101',
   'sub-0093',
   'sub-0012',
   'sub-0050',
   'sub-0046',
   'sub-0123',
   'sub-0045',
   'sub-0048',
   'sub-0021',
   'sub-0015',
   'sub-0028',
   'sub-0094',
   'sub-0095',
   'sub-0011',
   'sub-0122',
   'sub-0120',
   'sub-0087',
   'sub-0110',
   'sub-0089',
   'sub-0082',
   'sub-0029',
   'sub-0033',
   'sub-0083',
   'sub-0098',
   'sub-0113',
   'sub-0051',
   'sub-0112',
   'sub-0014',
   'sub-0020',
   'sub-0084',
   'sub-0096',
   'sub-0105',
   'sub-0017',
   'sub-0052',
   'sub-0005',
   'sub-0106',
   'sub-0080',
   'sub-0126',
   'sub-0100',
   'sub-0086',
   'sub-0117',
   'sub-0104',
   'sub-0092',
   'sub-0034',
   'sub-0044',
   'sub-0039',
   'sub-0107',
   'sub-0003',
   'sub-0009',
   'sub-0111',
   'sub-0127',
   'sub-0035',
   'sub-0036'],
  'valid': ['sub-0079',
   'sub-0031',
   'sub-0109',
   'sub-0085',
   'sub-0102',
   'sub-0090',
   'sub-0047',
   

In [115]:
import json
with open('../datasets/alexeev/alexeev_splits.json', 'w') as f:
    json.dump(new_folds, f)